In [9]:
import tkinter as tk
from tkinter import messagebox
import heapq
import random

# 定義迪士尼樂園的鄰接矩陣
# 使用字典來表示每個設施之間的距離
distances = {
    '入口': {'太空山': 10, '加勒比海盜': 15, '鬼屋': 20},
    '太空山': {'飛濺山': 20, '雷鳴山': 30, '星際大戰': 15, '小飛象': 25, '灰姑娘城堡': 10},
    '加勒比海盜': {'飛濺山': 35, '夢幻樂園': 25, '叢林巡航': 20},
    '鬼屋': {'雷鳴山': 20, '夢幻樂園': 30, '印第安納瓊斯': 15, '美女與野獸': 25},
    '飛濺山': {'米奇歡樂谷': 10, '泰山樹屋': 15, '旋轉木馬': 20},
    '雷鳴山': {'米奇歡樂谷': 15, '雪山飛車': 20, '彼得潘之旅': 30},
    '夢幻樂園': {'米奇歡樂谷': 20, '小小世界': 10},
    '米奇歡樂谷': {},
    '星際大戰': {'米奇歡樂谷': 25, '米奇交響樂團': 30},
    '叢林巡航': {'印第安納瓊斯': 10},
    '印第安納瓊斯': {'米奇歡樂谷': 25},
    '泰山樹屋': {'米奇歡樂谷': 20},
    '雪山飛車': {'米奇歡樂谷': 15},
    '小小世界': {'米奇歡樂谷': 15},
    '小飛象': {'夢幻樂園': 20},
    '米奇交響樂團': {'夢幻樂園': 25},
    '灰姑娘城堡': {'夢幻樂園': 15},
    '美女與野獸': {'夢幻樂園': 20},
    '旋轉木馬': {'夢幻樂園': 10},
    '彼得潘之旅': {'夢幻樂園': 15}
}

# 定義每個設施的期望值
# 期望值代表每個設施的吸引力分數
expected_values = {
    '入口': 0,
    '太空山': 8,
    '加勒比海盜': 6,
    '鬼屋': 9,
    '飛濺山': 7,
    '雷鳴山': 10,
    '夢幻樂園': 5,
    '米奇歡樂谷': 8,
    '星際大戰': 9,
    '叢林巡航': 6,
    '印第安納瓊斯': 8,
    '泰山樹屋': 5,
    '雪山飛車': 7,
    '小小世界': 6,
    '小飛象': 7,
    '米奇交響樂團': 8,
    '灰姑娘城堡': 7,
    '美女與野獸': 8,
    '旋轉木馬': 6,
    '彼得潘之旅': 9
}

# 定義每個設施的遊玩時間（以分鐘為單位）
play_times = {
    '入口': 0,
    '太空山': 30,
    '加勒比海盜': 45,
    '鬼屋': 20,
    '飛濺山': 25,
    '雷鳴山': 35,
    '夢幻樂園': 30,
    '米奇歡樂谷': 25,
    '星際大戰': 40,
    '叢林巡航': 30,
    '印第安納瓊斯': 35,
    '泰山樹屋': 20,
    '雪山飛車': 25,
    '小小世界': 30,
    '小飛象': 20,
    '米奇交響樂團': 30,
    '灰姑娘城堡': 25,
    '美女與野獸': 35,
    '旋轉木馬': 15,
    '彼得潘之旅': 25
}

# 定義每個設施的排隊時間範圍（以分鐘為單位）
# 這些時間會在程序運行時動態變化
queue_time_ranges = {
    '入口': (0, 0),
    '太空山': (5, 15),
    '加勒比海盜': (10, 20),
    '鬼屋': (5, 10),
    '飛濺山': (10, 15),
    '雷鳴山': (15, 25),
    '夢幻樂園': (10, 20),
    '米奇歡樂谷': (5, 10),
    '星際大戰': (20, 30),
    '叢林巡航': (10, 15),
    '印第安納瓊斯': (10, 20),
    '泰山樹屋': (5, 10),
    '雪山飛車': (10, 15),
    '小小世界': (5, 10),
    '小飛象': (5, 15),
    '米奇交響樂團': (10, 20),
    '灰姑娘城堡': (10, 15),
    '美女與野獸': (15, 25),
    '旋轉木馬': (5, 10),
    '彼得潘之旅': (10, 20)
}

# 初始化每個設施的排隊時間
# 使用隨機數生成器在指定範圍內生成初始排隊時間
queue_times = {ride: random.randint(*queue_time_ranges[ride]) for ride in queue_time_ranges}

# 設施的二維位置，用於在畫布上繪製
positions = {
    '入口': (50, 350),
    '太空山': (150, 250),
    '加勒比海盜': (300, 350),
    '鬼屋': (450, 300),
    '飛濺山': (600, 250),
    '雷鳴山': (750, 200),
    '夢幻樂園': (200, 150),
    '米奇歡樂谷': (350, 100),
    '星際大戰': (500, 50),
    '叢林巡航': (650, 100),
    '印第安納瓊斯': (800, 150),
    '泰山樹屋': (900, 200),
    '雪山飛車': (1050, 250),
    '小小世界': (1200, 300),
    '小飛象': (150, 100),
    '米奇交響樂團': (1000, 50),
    '灰姑娘城堡': (300, 50),
    '美女與野獸': (400, 50),
    '旋轉木馬': (700, 50),
    '彼得潘之旅': (900, 50)
}

# 使用Dijkstra演算法計算從起點到其他節點的最短距離
def dijkstra(graph, start):
    # 優先隊列
    queue = [(0, start)]
    heapq.heapify(queue)
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    shortest_path = {}

    while queue:
        (current_distance, current_node) = heapq.heappop(queue)

        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(queue, (distance, neighbor))
                shortest_path[neighbor] = current_node

    return distances, shortest_path

# 計算選擇路徑的總時間，包括遊玩時間和排隊時間
def calculate_total_time(route):
    total_time = 0
    for ride in route:
        total_time += play_times[ride] + queue_times[ride]
    return total_time

# 在畫布上繪製迪士尼樂園的設施和路徑
def plot_park(canvas, positions, selected_rides, route=None, clear=False):
    canvas.delete("all")
    for ride, pos in positions.items():
        color = 'blue' if ride in selected_rides else 'grey'
        canvas.create_oval(pos[0] - 10, pos[1] - 10, pos[0] + 10, pos[1] + 10, fill=color, outline="")
        canvas.create_text(pos[0], pos[1] - 20, text=ride)
    if route and not clear:
        for i in range(len(route) - 1):
            start_pos = positions[route[i]]
            end_pos = positions[route[i + 1]]
            canvas.create_line(start_pos[0], start_pos[1], end_pos[0], end_pos[1], fill='red', arrow=tk.LAST)
        for idx, ride in enumerate(route):
            pos = positions[ride]
            canvas.create_text(pos[0], pos[1] + 20, text=f"{idx + 1}", fill='red')

# 更新設施的排隊時間
def update_queue_times():
    for ride in queue_times:
        if ride != '入口':
            queue_times[ride] = random.randint(*queue_time_ranges[ride])

# 當點擊畫布時，選擇或取消選擇設施
def on_canvas_click(event):
    x, y = event.x, event.y
    for ride, pos in positions.items():
        if ride == '入口':
            continue
        if pos[0] - 10 <= x <= pos[0] + 10 and pos[1] - 10 <= y <= pos[1] + 10:
            if ride in selected_rides:
                selected_rides.remove(ride)
            else:
                selected_rides.add(ride)
            plot_park(canvas, positions, selected_rides)
            break

# 計算最佳路徑，並顯示結果
def calculate_best_route():
    if not selected_rides:
        messagebox.showinfo("提示", "請選擇至少一個遊樂設施")
        return

    update_queue_times()

    selected_rides_list = list(selected_rides)
    selected_rides_list.insert(0, '入口')  # 從入口開始
    distances_from_entrance, _ = dijkstra(distances, '入口')

    sorted_rides = sorted(selected_rides_list, key=lambda x: (distances_from_entrance[x], -expected_values[x]))
    route = sorted_rides
    total_time = calculate_total_time(route)

    result_text = "最佳路徑規劃：\n"
    for ride in route:
        result_text += f"{ride} - 距離: {distances_from_entrance[ride]}, 期望值: {expected_values[ride]}, 遊玩時間: {play_times[ride]} 分鐘, 排隊時間: {queue_times[ride]} 分鐘\n"
    result_text += f"\n總時間: {total_time} 分鐘"

    plot_park(canvas, positions, selected_rides, route)
    messagebox.showinfo("最佳路徑規劃", result_text)

# 清空路徑
def clear_route():
    global selected_rides
    selected_rides = set()
    plot_park(canvas, positions, selected_rides, clear=True)
    for ride, pos in positions.items():
        if ride != '入口':
            canvas.create_oval(pos[0] - 10, pos[1] - 10, pos[0] + 10, pos[1] + 10, fill='grey', outline="")


# 創建Tkinter主窗口
root = tk.Tk()
root.title("迪士尼樂園路徑規劃")

selected_rides = set()

# 創建畫布
canvas = tk.Canvas(root, width=1300, height=400, bg="white")
canvas.pack()

plot_park(canvas, positions, selected_rides)
canvas.bind("<Button-1>", on_canvas_click)

# 添加計算按鈕
btn_calculate = tk.Button(root, text="計算最佳路徑", command=calculate_best_route)
btn_calculate.pack(side=tk.LEFT, padx=10)

# 添加清空按鈕
btn_clear = tk.Button(root, text="清空路徑", command=clear_route)
btn_clear.pack(side=tk.LEFT, padx=10)

# 啟動主循環
root.mainloop()
